In [1]:
from pathlib import Path
import copy

import torch
import pyscf
import numpy as np

from cadft.utils import Grid
from cadft import CC_DFT_DATA, Mol

/home/dhem/anaconda3/envs/pyscf/lib/python3.11/site-packages/pyscf/dft/libxc.py:771: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [2]:
molecular = copy.deepcopy(Mol["Propane"])
molecular[0][1] += 0.16

dft2cc = CC_DFT_DATA(
    molecular,
    name="test",
    basis="cc-pvdz",
    if_basis_str=True,
)

mf = pyscf.scf.RHF(dft2cc.mol)
mf.kernel()
mycc = pyscf.cc.CCSD(mf)
mycc.kernel()

dm1_cc = mycc.make_rdm1(ao_repr=True)
e_cc = mycc.e_tot

mdft = pyscf.scf.RKS(dft2cc.mol)
mdft.xc = "b3lyp"
mdft.kernel()
e_dft = mdft.e_tot
dm1_dft = mdft.make_rdm1(ao_repr=True)

grids = Grid(dft2cc.mol)
coords = grids.coords
weights = grids.weights
ao_value = pyscf.dft.numint.eval_ao(dft2cc.mol, coords)

rho_dft = pyscf.dft.numint.eval_rho(dft2cc.mol, ao_value, dm1_dft)
rho_cc = pyscf.dft.numint.eval_rho(dft2cc.mol, ao_value, dm1_cc)

rho_dft_grids = grids.vector_to_matrix(rho_dft)
rho_cc_grids = grids.vector_to_matrix(rho_cc)
rho_cc_grids_pred = np.zeros_like(rho_cc_grids)

KeyboardInterrupt: 

In [3]:
from cadft.utils import ModelDict

Modeldict = ModelDict(100, 3, 1, "cuda")
Modeldict.load_model("2024-05-30-16-31-04")

Model loaded from checkpoints/checkpoint-ccdft-2024-05-30-16-31-04-100/1-10000.pth
Model loaded from checkpoints/checkpoint-ccdft-2024-05-30-16-31-04-100/2-10000.pth


In [4]:
ene_ = 0
for i_atom in range(rho_dft_grids.shape[0]):
    for i in range(rho_dft_grids.shape[1]):
        input_ = rho_dft_grids[i_atom, i, :]
        input_gpu = torch.tensor(input_, dtype=torch.float64).to("cuda")
        output_ = Modeldict.model_dict["1"](input_gpu).detach().cpu().numpy()
        rho_cc_grids_pred[i_atom, i, :] = output_ / 1000 + input_
        output_ = Modeldict.model_dict["2"](input_gpu).detach().cpu().numpy()
        ene_ += output_[0]

In [5]:
np.sum(np.abs(grids.matrix_to_vector(rho_cc_grids_pred - rho_cc_grids) * weights))

0.16013346968448577

In [6]:
e_dft * 1000 + ene_, e_cc * 1000

(-118750.29532288105, -118752.90660467514)